<a href="https://colab.research.google.com/github/nirav3/data690_nirav/blob/main/assignments/assignment_6/assignment_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>